In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [1]:
%reload_kedro

[09/17/24 20:22:12] INFO     Resolved project path as:                                              ]8;id=505448;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=363129;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py#175\175]8;;\
                             /home/saraiva/studies/ia/magalu/gu-ml/model-kedro.                                    
                             To set a different path, run '%reload_kedro <project_root>'                           

                    WARNING  /home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/framewo ]8;id=804485;file:///usr/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=416531;file:///usr/lib/python3.12/warnings.py#110\110]8;;\
                             rk/project/__init__.py:326: UserWarning: The                                          
                             'model_kedro.pipelines.training_model' module does not expose a                       
                             'create_pipeline' function, so no pipelines defined therein will be                   
                             returned by 'find_pipelines'.                                                         
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=644789;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=81420;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project model_kedro                                              ]8;id=965881;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=41805;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=125099;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=299151;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py#142\142]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=26648;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=199463;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/ipython/__init__.py#148\148]8;;\

In [2]:
catalog


{'train_df': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('/home/saraiva/studies/ia/magalu/gu-ml/model-kedro/data/challenge_train.csv'), "
             "protocol='file', load_args={}, save_args={'index': False})",
 'parameters': "kedro.io.memory_dataset.MemoryDataset(data='<dict>')"}

In [3]:
train_df = catalog.load("train_df")

[09/17/24 20:22:17] INFO     Loading data from train_df (CSVDataset)...                         ]8;id=486610;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=976324;file:///home/saraiva/pyvenv/magalu/lib/python3.12/site-packages/kedro/io/data_catalog.py#539\539]8;;\

In [6]:
train_df['mana_group'] = pd.cut(train_df['mana'], 5, labels=[0,1,2,3,4])
train_df['attack_group'] = pd.cut(train_df['attack'], 5, labels=[0,1,2,3,4])
train_df['health_group'] = pd.cut(train_df['attack'], 5, labels=[0,1,2,3,4])
gods_c = {'neutral': 0,
          'light': 1,
          'nature': 2,
          'deception': 3,
          'death': 4,
          'war': 5,
          'magic': 6}
train_df['god_categories'] = train_df['god'].map(lambda x: gods_c.get(x, 7))
type_c = {'creature': 0,
          'spell': 1,
          'weapon': 2,
          'god power': 3}
train_df['type_categories'] = train_df['type'].map(lambda x: type_c.get(x, 4))
train_df['strategy_categories'] = train_df['strategy'].map(lambda x: 0 if x == 'early' else 1)

In [7]:
X = train_df[['type_categories', 'god_categories', 'mana_group', 'attack_group', 'health_group', 'mana', 'health', 'attack']]
y = train_df['strategy_categories']

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X[['type_categories', 'god_categories', 'mana_group', 'attack_group', 'health_group']], 
    y, test_size=0.2, random_state=42)

In [8]:
def generate_model(train_df):
    import pandas as pd
    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    train_df['mana_group'] = pd.cut(train_df['mana'], 5, labels=[0,1,2,3,4])
    train_df['attack_group'] = pd.cut(train_df['attack'], 5, labels=[0,1,2,3,4])
    train_df['health_group'] = pd.cut(train_df['attack'], 5, labels=[0,1,2,3,4])
    gods_c = {'neutral': 0,
          'light': 1,
          'nature': 2,
          'deception': 3,
          'death': 4,
          'war': 5,
          'magic': 6}
    train_df['god_categories'] = train_df['god'].map(lambda x: gods_c.get(x, 7))
    type_c = {'creature': 0,
          'spell': 1,
          'weapon': 2,
          'god power': 3}
    train_df['type_categories'] = train_df['type'].map(lambda x: type_c.get(x, 4))
    train_df['strategy_categories'] = train_df['strategy'].map(lambda x: 0 if x == 'early' else 1)
    X = train_df[['type_categories', 'god_categories', 'mana_group', 'attack_group', 'health_group', 'mana', 'health', 'attack']]
    y = train_df['strategy_categories']
    X_train, X_valid, y_train, y_valid = train_test_split(
    X[['type_categories', 'god_categories', 'mana_group', 'attack_group', 'health_group']], 
    y, test_size=0.2, random_state=42)
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=['type_categories', 'god_categories', 'mana_group', 'health_group', 'attack_group'])
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 11,
    'learning_rate': 0.05,
    }
    num_round = 100
    lgb_model = lgb.train(params, train_data, num_round,
                valid_sets=[train_data, valid_data])
    return lgb_model

In [9]:
lgb_model_t = generate_model(train_df)

[LightGBM] [Info] Number of positive: 290, number of negative: 340
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068804 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28
[LightGBM] [Info] Number of data points in the train set: 630, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.460317 -> initscore=-0.159065
[LightGBM] [Info] Start training from score -0.159065
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [12]:
lgb_model_t